## Machine Learning Exercise 3: Bias and Variance

**Bias** refers to the error introduced by approximating a complex real-world problem with a simplified model, while **variance** refers to the model's sensitivity to fluctuations in the training data. A linear regression model has high bias and low variance; it makes strong assumptions about the data (linearity) but is stable across different datasets. If these strong assumptions are not correct, there will be places where it systematically overestimates or underestimates. In contrast, a decision tree model has low bias and high variance;it can capture complex patterns but is prone to overfitting, especially if deep and unpruned. This means that it can start to memorize the training data rather than capturing patterns that generalize.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import itables
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error, confusion_matrix, classification_report, roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from itertools import product


1. Fit a linear regression model to the housing data, using sqft_living to predict price. Check the mean squared error on the training data and the test data.

In [3]:
house_data = pd.read_csv("./data/kc_house_data.csv")
predictors = ["sqft_living"]
target = "price"

print(type(predictors))

X = house_data[predictors]
y = house_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

print(f"mse train = {mean_squared_error(y_true = y_train, y_pred = linreg.predict(X_train))}")
print("mean squared error test: ", mean_squared_error(y_true = y_test, y_pred = y_pred))

print("R^2: ", r2_score(y_test, y_pred))

<class 'list'>
mse train = 65713942542.23366
mean squared error test:  74509993356.49603
R^2:  0.48388319278201475



2. Repeat this but with a [DecisionTreeRegresor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). Again check the mean squared error on the training data and the test data. How does what you see differ from the linear regression model?


In [4]:
X = house_data[predictors]
y = house_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

dtreer = DecisionTreeRegressor()
dtreer.fit(X_train, y_train)
y_pred = dtreer.predict(X_test)

print(f"mse train = {mean_squared_error(y_true = y_train, y_pred = dtreer.predict(X_train))}")
print("mean squared error: ", mean_squared_error(y_test, y_pred))



mse train = 48410745078.45624
mean squared error:  79044952597.92511


the MSE is higher for the decision tree regressor. 


One way of avoiding overfitting is by restricting the flexibility of the model. We can do this with a decision tree by restricting the number of splits that it can perform. 


3. Fit a DecisionTreeRegressor where you restrict the max_depth to 5. Again check the mean squared error on the training data and the test data. What do you notice now?

In [5]:
X = house_data[predictors]
y = house_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

dtreer = DecisionTreeRegressor(max_depth=5)
dtreer.fit(X_train, y_train)
y_pred = dtreer.predict(X_test)

print(f"mse train = {mean_squared_error(y_true = y_train, y_pred = dtreer.predict(X_train))}")
print("mean squared error: ", mean_squared_error(y_test, y_pred))


mse train = 56030892751.02288
mean squared error:  73264250436.88351


the MSE went down and r^2 went up. better fit.

When working with machine learning models, we often have to balance bias and variance. This is called the [bias-variance tradeoff](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff). One method of this is through [regularization](https://www.ibm.com/think/topics/regularization), where we restrict the complexity of the model, adding some bias but reducing the variance, which can lead to a lower mean squared error on the test set.

Lasso and ridge regression do this by adding a penalty term based on the size of the coefficients. Smaller coefficients means that the model has less flexibility. The neat thing about these types of models is that they determine how to allocate the coefficients automatically as part of the model fitting process, so we can start with a large set of potential predictors and allow the model fitting to determine which ones to focus on.

For the next part of the exercise, we'll see how we can add complexity to our model but control the complexity through regularization.


4. So far, we've only been predicting based off of the square footage of living space. Fit a new linear regression model using all variables besides id, date, price, and zipcode. How well does this model perform on the test set compared to the model with just square footage of living space?


In [6]:
predictors = list(house_data.drop(columns=["id", "date", "price", "zipcode"]).columns)
target = "price"

X = house_data[predictors]
y = house_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))



mean squared error:  44093475276.57266


this model does better than the model using just the sqftage. 

5. Try fitting a lasso and ridge model. Becuase lasso and ridge have penalty terms based on the size of the coefficients, and the size of the coefficients depends on the scale of the variable, you'll want to scale the features first so that they are on comparable scales. Create a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) object where the first step is applying a [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and the second step is either a lasso or ridge model. Because these models have a hyperparameter controlling regularization strength, you'll want to use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) models, which will select values for the regularization strength using cross-validation.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

steps = [
    ("scale", StandardScaler()),
    ("lasso", LassoCV())
]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  44128929521.62934


In [8]:
steps = [
    ("scale", StandardScaler()),
    ("ridge", RidgeCV())
]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


print("mean squared error: ", mean_squared_error(y_test, y_pred))
print("root mean squared error: ", root_mean_squared_error(y_test, y_pred))
print("mean absolute error: ", mean_absolute_error(y_test, y_pred))
print("mean absolute percentage error: ", mean_absolute_percentage_error(y_test, y_pred))
print("R^2: ", r2_score(y_test, y_pred))

mean squared error:  44094763932.52806
root mean squared error:  209987.53280261203
mean absolute error:  127878.04498292944
mean absolute percentage error:  0.25290584991142046
R^2:  0.694563806132684


You likely didn't see much difference between the regular linear regression model and the lasso or ridge model. Let's see what happens if we add more complexity through feature interactions. We can capture more complex relationships between the predictor variables and the target variable by multiplying the predictors together before fitting the model. For example, the interaction between sqft_living and bedrooms will let the model capture if the impact of square footage depends on the number of bedrooms.


6. Add [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to your pipeline after the standard scaler. Try using degree 2 features. How does this change the performance of the regular linear regression model, the lasso model, and the ridge model? 

The lasso penalty tends to cause some coeffients to zero out, so it can be viewed as a method of automatic feature selection.


In [9]:
steps = [
    ("scale", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2)),
    ("linreg", LinearRegression())
]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


print("mean squared error: ", mean_squared_error(y_test, y_pred))


mean squared error:  30926927395.20743


In [10]:
steps = [
    ("scale", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2)),
    ("lasso", LassoCV())
]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


print("mean squared error: ", mean_squared_error(y_test, y_pred))


mean squared error:  30414023904.742867


In [11]:
steps = [
    ("scale", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2)),
    ("ridge", RidgeCV())
]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


print("mean squared error: ", mean_squared_error(y_test, y_pred))


mean squared error:  30933832194.40976



7. Look at the set of coefficients for the lasso model. What percentage of the coefficients are zero? What are the largest non-zero coefficients?


In [15]:
steps = [
    ("scale", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2)),
    ("lasso", LassoCV())
]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

lasso_model = pipeline.named_steps['lasso']
coefficients = lasso_model.coef_
print(f"%-coefs zero = {100 - np.count_nonzero(coefficients)/len(coefficients)*100}")

coefficients = pd.DataFrame({
    'variable': pipeline[:-1].get_feature_names_out(),
    'coefficient': pipeline['lasso'].coef_
})

print("mean squared error: ", mean_squared_error(y_test, y_pred))

print((coefficients['coefficient'] != 0).mean())
print(coefficients.sort_values('coefficient'))

%-coefs zero = 48.538011695906434
mean squared error:  25135804989.076466
0.5146198830409356
                   variable   coefficient
161                   lat^2 -41215.451933
154  yr_built sqft_living15 -25451.428122
15                     long -23944.349431
132              grade long -21063.775277
12                 yr_built -18549.691563
..                      ...           ...
16            sqft_living15  38574.730072
57        sqft_living grade  39077.771105
14                      lat  70301.434205
3               sqft_living  95077.950449
9                     grade  96953.631743

[171 rows x 2 columns]


8. A new hyperparameter that we have is the degree of the polynomial we're using. So that we're not overfitting to the test set, we need to use cross-validation to select this value. Set up a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to try out polynomial degrees from 1 to 3 and to try LinearRegression, LassoCV, and RidgeCV models. Use 'neg_mean_squared_error' as the error_score. Which combination does it find does the best? 

In [16]:
predictors = list(house_data.drop(columns=["id", "date", "price", "zipcode"]).columns)
target = "price"

X = house_data[predictors]
y = house_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 321)

pipeline = Pipeline([
        ("scale", StandardScaler()),
        ("poly", PolynomialFeatures()),
        ("model", LinearRegression())
])

param_grid = {
    "poly__degree": [1, 2, 3],
    "model": [LinearRegression(), LassoCV(max_iter = 10000), RidgeCV()],
}

gs = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    # n_jobs=-1,
    verbose=2
)

gs.fit(X_train, y_train)
print(f"best params = {gs.best_params_}")
print(f"best score = {gs.best_score_}")
print(f"best pipeline = {gs.best_estimator_}")
best_model = gs.best_estimator_.named_steps['model']
print(f"best model = {best_model}")



Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ...........model=LinearRegression(), poly__degree=1; total time=   0.0s
[CV] END ...........model=LinearRegression(), poly__degree=1; total time=   0.0s
[CV] END ...........model=LinearRegression(), poly__degree=1; total time=   0.0s
[CV] END ...........model=LinearRegression(), poly__degree=1; total time=   0.0s
[CV] END ...........model=LinearRegression(), poly__degree=1; total time=   0.0s
[CV] END ...........model=LinearRegression(), poly__degree=2; total time=   0.1s
[CV] END ...........model=LinearRegression(), poly__degree=2; total time=   0.1s
[CV] END ...........model=LinearRegression(), poly__degree=2; total time=   0.1s
[CV] END ...........model=LinearRegression(), poly__degree=2; total time=   0.1s
[CV] END ...........model=LinearRegression(), poly__degree=2; total time=   0.1s
[CV] END ...........model=LinearRegression(), poly__degree=3; total time=   0.9s
[CV] END ...........model=LinearRegression(), pol

/Users/drewrichard/Documents/projects/nss/nss_projects/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/drewrichard/Documents/projects/nss/nss_projects/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/drewrichard/Documents/projects/nss/nss_projects/.venv/lib/python3.11/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^

best params = {'model': LassoCV(max_iter=10000), 'poly__degree': 2}
best score = -29157768309.902992
best pipeline = Pipeline(steps=[('scale', StandardScaler()), ('poly', PolynomialFeatures()),
                ('model', LassoCV(max_iter=10000))])
best model = LassoCV(max_iter=10000)


** If you've reached this point, let your instructors know so that they can check in with you. **


Stretch Goals:

1. With home prices, there are some extremely large values for price, which can overly-influence the mean squared error calculation. See what happens if you optimize for mean absolute error instead. Alternatively, try using a [TransformedTargetRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html) to predict the log of price instead of the raw price.


In [ ]:
predictors = list(house_data.drop(columns=["id", "date", "price", "zipcode"]).columns)
target = "price"
X = house_data[predictors]
y = house_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 321)
pipeline = Pipeline(
    [
        ("normalize", StandardScaler()),
        ("poly", PolynomialFeatures()),
        ("model", LassoCV())
    ]
)
param_grid = {
    "poly__degree": [1, 2, 3],
    "model": [LassoCV(max_iter = 10000), RidgeCV()],
}
gs = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=5,
    n_jobs=-1
    # verbose=2,
    # pre_dispatch='2*n_jobs'
)
gs.fit(X_train, y_train)
print(f"best params = {gs.best_params_}")
print(f"best score = {gs.best_score_}")
print(f"best pipeline = {gs.best_estimator_}")
best_model = gs.best_estimator_.named_steps['model']
print(f"best model = {best_model}")
# mean absolute error brings the error term to the same scale as our target, so it's easier to understand. an error of around $101K is still pretty large, but very bad.


/Users/drewrichard/Documents/projects/nss/nss_projects/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


best params = {'model': LassoCV(max_iter=10000), 'poly__degree': 3}
best score = -101897.9617589312
best pipeline = Pipeline(steps=[('normalize', StandardScaler()),
                ('poly', PolynomialFeatures(degree=3)),
                ('model', LassoCV(max_iter=10000))])
best model = LassoCV(max_iter=10000)


In [ ]:

#TransformedTargetRegressor
predictors = list(house_data.drop(columns=["id", "date", "price", "zipcode"]).columns)
target = "price"
X = house_data[predictors]
y = house_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 321)
pipeline = Pipeline(
    [
        ("normalize", StandardScaler()),
        ("poly", PolynomialFeatures()),
        ("model", LassoCV()),
    ]
)
param_grid = {
    "poly__degree": [1, 2, 3],
    "model": [
        LassoCV(max_iter = 10000),
        RidgeCV(),
        TransformedTargetRegressor(
            regressor = LassoCV(max_iter = 10000),
            func=np.log,
            inverse_func=np.exp
        ),
        TransformedTargetRegressor(
            regressor = RidgeCV(),
            func=np.log,
            inverse_func=np.exp
        )
    ],
}
gs = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=5,
    n_jobs=-1
    # verbose=2,
    # pre_dispatch='2*n_jobs'
)
gs.fit(X_train, y_train)
print(f"best params = {gs.best_params_}")
print(f"best score = {gs.best_score_}")
print(f"best pipeline = {gs.best_estimator_}")
best_model = gs.best_estimator_.named_steps['model']
print(f"best model = {best_model}")


/Users/drewrichard/Documents/projects/nss/nss_projects/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


best params = {'model': TransformedTargetRegressor(func=<ufunc 'log'>, inverse_func=<ufunc 'exp'>,
                           regressor=LassoCV(max_iter=10000)), 'poly__degree': 3}
best score = -91578.46123978362
best pipeline = Pipeline(steps=[('normalize', StandardScaler()),
                ('poly', PolynomialFeatures(degree=3)),
                ('model',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 'exp'>,
                                            regressor=LassoCV(max_iter=10000)))])
best model = TransformedTargetRegressor(func=<ufunc 'log'>, inverse_func=<ufunc 'exp'>,
                           regressor=LassoCV(max_iter=10000))



**Bonus Exercise** We've seen how a decision tree model has move flexibility, which mean higher variance compared to a linear regression model. One way of understanding variance is that variance describes how sensitive the model is to the training data. A model with high variance can produce vastly different predictions when trained on different subsets.

In this bonus exercise, you'll get to see this in action.

Generate 1000 different linear regression fits, where you only use sqft_living as the predictor variable. For each fit, choose a random sample from the full dataset of size 1000. 
Using the np.linspace function, generate a grid of 100 equally-spaced points between 500 and 3000 and generate predictions on those points. Plot the mean prediction, the 5th percentile, and the 95th percentile for the predictions from these thousand models. Repeat this for a decision tree model. Then do it for a decision tree model with a max_depth of 5.

How do these predictions differ? Which have the most variability?
